In [43]:
from langchain_community.document_loaders import TextLoader as TL
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import os

In [35]:
from dotenv import load_dotenv

load_dotenv()


True

In [36]:
print(os.getcwd())

c:\Users\jaypu\Documents\TU Ilmenau\Kaggle\book-recommender


In [3]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [4]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep="\n",
                                   index=False,
                                   header=False)

In [ ]:
books

In [37]:
raw_documents = TL("tagged_description.txt", encoding='utf-8').load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 0
Created a chunk of size 1214, which is longer than the specified 0
Created a chunk of size 373, which is longer than the specified 0
Created a chunk of size 309, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 960, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 843, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1088, which is longer than the specified 0
Created a chunk of size 1189, which is longer than the specified 0
Created a chunk of size 304, which is longer than the specified 0
Create

In [38]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [48]:
# Start building vector database
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Use a small, fast, high-quality model
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db_books = Chroma.from_documents(documents, embedding=embedding)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [49]:
query = " A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(metadata={'source': 'tagged_description.txt'}, page_content="9780786808397 Introduce your baby to birds, cats, dogs, and babies through fine art, illu

In [ ]:
# we want recommendation for book titles, find a way to get isbn
# take docs, exttact page content, keep only first element and strip
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [51]:
# function to bundle the recommendations
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [52]:
retrieve_semantic_recommendations(" A book about science")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036 Barbara Kingsolver's fifth novel...
393,9780061238239,0061238236,The End of Days,Zecharia Sitchin,History,http://books.google.com/books/content?id=EIBlj...,A conclusion to the Earth Chronicles series br...,2007.0,4.06,336.0,470.0,The End of Days: Armageddon and Prophecies of ...,9780061238239 A conclusion to the Earth Chroni...
549,9780131871656,013187165X,Astronomy,Eric Chaisson;Stephen McMillan,Mathematics,http://books.google.com/books/content?id=1O00A...,This introduction to astronomy features an exc...,2006.0,3.85,499.0,153.0,Astronomy: a beginner's guide to the universe,9780131871656 This introduction to astronomy f...
635,9780140296617,0140296611,The Periodic Table,Primo Levi;Raymond Rosenthal,"Authors, Italian",http://books.google.com/books/content?id=TOo-P...,An extraordinary work in which each of the 21 ...,2001.0,4.20,194.0,98.0,The Periodic Table,9780140296617 An extraordinary work in which e...
937,9780156932509,0156932504,The Uses of Literature,Italo Calvino,Literary Criticism,http://books.google.com/books/content?id=INlwz...,This essay collection discusses literature in ...,1987.0,4.07,341.0,887.0,The Uses of Literature: Essays,9780156932509 This essay collection discusses ...
991,9780192862099,019286209X,The Origins of Life,John Maynard Smith;Eörs Szathmáry,Science,http://books.google.com/books/content?id=nHDbB...,'I can recommend this book as a thoroughly int...,2000.0,4.11,192.0,41.0,The Origins of Life: From the Birth of Life to...,9780192862099 'I can recommend this book as a ...
1026,9780201408256,0201408252,Six Easy Pieces,Richard P. Feynman,Science,http://books.google.com/books/content?id=xerju...,Richard P. Feynman (1918–1988) was widely reco...,1996.0,4.20,176.0,160.0,Six Easy Pieces: Essentials Of Physics By Its ...,9780201408256 Richard P. Feynman (1918–1988) w...
1060,9780231054751,0231054750,Genetics and the Origin of Species,Theodosius Dobzhansky,Science,http://books.google.com/books/content?id=s7igy...,Featuring an introduction by Stephen Jay Gould...,1982.0,4.15,364.0,24.0,Genetics and the Origin of Species,9780231054751 Featuring an introduction by Ste...
1255,9780312423131,0312423136,Galatea 2.2,Richard Powers,Fiction,http://books.google.com/books/content?id=ttcEu...,"The protagonist, a Humanist-in-Residence at th...",2004.0,3.72,329.0,2063.0,Galatea 2.2: A Novel,"9780312423131 The protagonist, a Humanist-in-R..."
1271,9780312852535,0312852533,The Humanoids,Jack Williamson,Fiction,http://books.google.com/books/content?id=vPSl0...,A classic science fiction novel features human...,1996.0,3.75,299.0,880.0,The Humanoids: A Novel,9780312852535 A classic science fiction novel ...
